# Basic download of ECMWF data using the native MARS client

We will download some sample data using the <a href="https://confluence.ecmwf.int/x/ERSYAg">MARS</a> client on the European Weather Cloud. We can typically do this on the command line, but we use the magic of Jupyter notebooks for an easier to follow narrative

## Configuring MARS Access

First of all, if not done already, make sure you have the right credentials configured in your instance.
1. Go to <a href="https://api.ecmwf.int/v1/key/">https://api.ecmwf.int/v1/key/</a>
2. Copy the contents of the .ecmwfapirc
3. Put them into ~/.ecmwfapirc. You can do it on a terminal, or directly from the notebook with:

In [1]:
%%writefile ~/.ecmwfapirc.demo
{
    "url"   : "https://api.ecmwf.int/v1",
    "key"   : "##############################",
    "email" : "################"
}

Overwriting /home/usxa/.ecmwfapirc.demo


<div class="alert alert-block alert-info"> <b>Connectivity remarks</b> <p>When running at ECMWF, make sure your instance has a public IP before running MARS. If using Floating IP, MARS client will not be able to figure out what it is so you will need to set it up explicitly.</p></div>

In [2]:
#%env MARS_DHS_LOCALHOST=$(curl https://ipinfo.io/ip)

## Running the simplest mars request
In order to test the access to MARS, we can run the following request

In [3]:
! echo "retrieve, target=first.grib" | mars

mars - INFO   - 20240419.160123 - Welcome to MARS
mars - INFO   - 20240419.160123 - MARS_HOME=/home/usxa/.marsrc/home
mars - INFO   - 20240419.160123 - MARS Client build stamp: 20240410103252
mars - INFO   - 20240419.160123 - MARS Client bundle version: 6.33.18.0
mars - INFO   - 20240419.160123 - MARS Client version: 20240410
mars - INFO   - 20240419.160123 - package mars-client version: 6.33.18
mars - INFO   - 20240419.160123 - package mir version: 1.19.1
mars - INFO   - 20240419.160123 - package odc version: 1.5.0
mars - INFO   - 20240419.160123 - package fdb version: 5.11.30
mars - INFO   - 20240419.160123 - package metkit version: 1.11.7
mars - INFO   - 20240419.160123 - package eckit version: 1.25.2
mars - INFO   - 20240419.160123 - package eccodes version: 2.34.1
retrieve, target=first.grib

mars - INFO   - 20240419.160123 - Processing request 1

RETRIEVE,
    CLASS      = OD,
    TYPE       = AN,
    STREAM     = OPER,
    EXPVER     = 0001,
    REPRES     = SH,
    LEVTYPE    =

We can quickly inspect the contents of 

In [4]:
!grib_ls first.grib

first.grib
edition      centre       typeOfLevel  level        dataDate     stepRange    dataType     shortName    packingType  gridType     
1            ecmf         isobaricInhPa  1000         20240418     0            an           z            spectral_complex  sh          
1            ecmf         isobaricInhPa  850          20240418     0            an           z            spectral_complex  sh          
1            ecmf         isobaricInhPa  700          20240418     0            an           z            spectral_complex  sh          
1            ecmf         isobaricInhPa  500          20240418     0            an           z            spectral_complex  sh          
1            ecmf         isobaricInhPa  400          20240418     0            an           z            spectral_complex  sh          
1            ecmf         isobaricInhPa  300          20240418     0            an           z            spectral_complex  sh          
6 of 6 messages in first.grib

6 of 

In [5]:
ls -l first.grib

-rw-r--r-- 1 usxa usxa 19682640 Apr 19 16:01 first.grib


We can also use the command line GRIB tools, part of the <a href="https://confluence.ecmwf.int/display/ECC">ECMWF ecCodes Library</a>, to quickly inspect the contents of the files retrieved:

In [6]:
!grib_ls first.grib

first.grib
edition      centre       typeOfLevel  level        dataDate     stepRange    dataType     shortName    packingType  gridType     
1            ecmf         isobaricInhPa  1000         20240418     0            an           z            spectral_complex  sh          
1            ecmf         isobaricInhPa  850          20240418     0            an           z            spectral_complex  sh          
1            ecmf         isobaricInhPa  700          20240418     0            an           z            spectral_complex  sh          
1            ecmf         isobaricInhPa  500          20240418     0            an           z            spectral_complex  sh          
1            ecmf         isobaricInhPa  400          20240418     0            an           z            spectral_complex  sh          
1            ecmf         isobaricInhPa  300          20240418     0            an           z            spectral_complex  sh          
6 of 6 messages in first.grib

6 of 

## A more realistic example

We can explore the MARS archive through the <a href="https://apps.ecmwf.int/mars-catalogue/">Mars Catalogue</a>. It is also the best way to craft your MARS requests to get the data you are after.

In this example, we will try to get 2-meter temperature data from HRES forecast from 2000-01-01 12 run at steps 3, 6, 9 and 12.

<a href="https://apps.ecmwf.int/mars-catalogue/?stream=oper&levtype=sfc&time=12%3A00%3A00&expver=1&month=jan&year=2000&date=2000-01-01&type=fc&class=od">Here</a> you can find the direct link to the relevant page in the MARS Catalogue for that information.

You can select the right elements in the menus, and click on "View MARS request" to get your starting point.

The request given should look similar to this one:
```retrieve,
class=od,
date=2000-01-01,
expver=1,
levtype=sfc,
param=167.128,
step=3/6/9/12,
stream=oper,
time=12:00:00,
type=fc,
target="output"
```

We can customise the target name, and also add additional postprocessing options such as the area or the grid we want, and write it to a request file:

In [7]:
%%writefile real.req
retrieve,
class=od,
date=2000-01-01,
expver=1,
levtype=sfc,
param=167.128,
step=3/6/9/12,
stream=oper,
time=12:00:00,
type=fc,
area=E,
grid=1/1,
target="real.grib"

Overwriting real.req


We can now pass that request to MARS to retrieve the data:

In [8]:
!mars real.req

mars - INFO   - 20240419.160124 - Welcome to MARS
mars - INFO   - 20240419.160124 - MARS_HOME=/home/usxa/.marsrc/home
mars - INFO   - 20240419.160124 - MARS Client build stamp: 20240410103252
mars - INFO   - 20240419.160124 - MARS Client bundle version: 6.33.18.0
mars - INFO   - 20240419.160124 - MARS Client version: 20240410
mars - INFO   - 20240419.160124 - package mars-client version: 6.33.18
mars - INFO   - 20240419.160124 - package mir version: 1.19.1
mars - INFO   - 20240419.160124 - package odc version: 1.5.0
mars - INFO   - 20240419.160124 - package fdb version: 5.11.30
mars - INFO   - 20240419.160124 - package metkit version: 1.11.7
mars - INFO   - 20240419.160124 - package eckit version: 1.25.2
mars - INFO   - 20240419.160124 - package eccodes version: 2.34.1
retrieve,
class=od,
date=2000-01-01,
expver=1,
levtype=sfc,
param=167.128,
step=3/6/9/12,
stream=oper,
time=12:00:00,
type=fc,
area=E,
grid=1/1,
target="real.grib"

mars - INFO   - 20240419.160124 - Processing request 1


Let's inspect the contents of the retrieved file, and ensure they match our request:

In [9]:
!grib_ls -m real.grib

real.grib
domain      levtype     date        time        step        param       class       type        stream      expver      
g           sfc         20000101    1200        3           167.128     od          fc          oper        0001       
g           sfc         20000101    1200        6           167.128     od          fc          oper        0001       
g           sfc         20000101    1200        9           167.128     od          fc          oper        0001       
g           sfc         20000101    1200        12          167.128     od          fc          oper        0001       
4 of 4 messages in real.grib

4 of 4 total messages in 1 files


We can also see the grid information in the geography namespace of ecCodes:

In [10]:
!grib_ls -n geography real.grib

real.grib
bitmapPresent                       latitudeOfFirstGridPointInDegrees   longitudeOfFirstGridPointInDegrees  latitudeOfLastGridPointInDegrees    longitudeOfLastGridPointInDegrees   iScansNegatively                    jScansPositively                    jPointsAreConsecutive               jDirectionIncrementInDegrees        iDirectionIncrementInDegrees        gridType                            
0                                   73                                  -27                                 33                                  45                                  0                                   0                                   0                                   1                                   1                                   regular_ll                         
0                                   73                                  -27                                 33                                  45                                  0                

## A bigger example

We can also see how a bigger request would perform. In this example, we will try to get a few GB of ENS data 

In [11]:
%%writefile big.req
retrieve,
class=od,
date=-1,
expver=1,
levtype=sfc,
number=1/to/50,
param=167.128,
step=0/to/24/by/6,
stream=enfo,
time=00:00:00,
type=pf,
target="big.grib"

Overwriting big.req


In [12]:
!mars big.req

mars - INFO   - 20240419.160125 - Welcome to MARS
mars - INFO   - 20240419.160125 - MARS_HOME=/home/usxa/.marsrc/home
mars - INFO   - 20240419.160125 - MARS Client build stamp: 20240410103252
mars - INFO   - 20240419.160125 - MARS Client bundle version: 6.33.18.0
mars - INFO   - 20240419.160125 - MARS Client version: 20240410
mars - INFO   - 20240419.160125 - package mars-client version: 6.33.18
mars - INFO   - 20240419.160125 - package mir version: 1.19.1
mars - INFO   - 20240419.160125 - package odc version: 1.5.0
mars - INFO   - 20240419.160125 - package fdb version: 5.11.30
mars - INFO   - 20240419.160125 - package metkit version: 1.11.7
mars - INFO   - 20240419.160125 - package eckit version: 1.25.2
mars - INFO   - 20240419.160125 - package eccodes version: 2.34.1
retrieve,
class=od,
date=-1,
expver=1,
levtype=sfc,
number=1/to/50,
param=167.128,
step=0/to/24/by/6,
stream=enfo,
time=00:00:00,
type=pf,
target="big.grib"

mars - INFO   - 20240419.160125 - Processing request 1

RETRIE